In [12]:
# Importing necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np

# Step 1: Load and preprocess the data
def load_and_preprocess_data():
    # Load the Iris dataset
    iris = load_iris()
    X = iris.data  # Features
    y = iris.target  # Labels (Classes)

    # One-hot encoding of the labels
    encoder = OneHotEncoder(sparse_output=False)
    y_onehot = encoder.fit_transform(y.reshape(-1, 1))

    # Splitting data into train and test sets (80% train, 20% test)
    X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, random_state=42)

    # Standardizing the features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Converting to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

    return X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor

# Step 2: Define the neural network model
class IrisNet(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(IrisNet, self).__init__()
        # First hidden layer
        self.fc1 = nn.Linear(input_size, hidden_size1)
        # Second hidden layer
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        # Output layer
        self.fc3 = nn.Linear(hidden_size2, output_size)
    
    def forward(self, x):
        # Apply ReLU activations after each layer except the last
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        # Apply softmax for multi-class classification at the output layer
        x = torch.softmax(self.fc3(x), dim=1)
        return x

# Step 3: Train the model
def train_model(model, X_train, y_train, epochs=50, learning_rate=0.01):
    # Loss function (cross-entropy for classification)
    criterion = nn.CrossEntropyLoss()
    # Optimizer (Adam)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(epochs):
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(X_train)
        # Calculate the loss
        loss = criterion(outputs, torch.max(y_train, 1)[1])

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

# Step 4: Evaluate the model
def evaluate_model(model, X_test, y_test):
    with torch.no_grad():  # Disable gradient calculation
        outputs = model(X_test)
        predicted = torch.argmax(outputs, 1)
        correct = torch.sum(predicted == torch.argmax(y_test, 1)).item()
        accuracy = correct / y_test.shape[0]
        print(f'Test Accuracy: {accuracy * 100:.2f}%')

# Step 5: Putting it all together
if __name__ == "__main__":
    # Load data
    X_train, X_test, y_train, y_test = load_and_preprocess_data()

    # Define model
    input_size = X_train.shape[1]
    hidden_size1 = 10
    hidden_size2 = 8
    output_size = y_train.shape[1]
    model = IrisNet(input_size, hidden_size1, hidden_size2, output_size)

    # Train model
    train_model(model, X_train, y_train, epochs=50, learning_rate=0.01)

    # Evaluate model
    evaluate_model(model, X_test, y_test)


Epoch [10/50], Loss: 0.9927
Epoch [20/50], Loss: 0.8593
Epoch [30/50], Loss: 0.7755
Epoch [40/50], Loss: 0.7103
Epoch [50/50], Loss: 0.6510
Test Accuracy: 100.00%
